In [6]:
!pip install matplotlib

     |████████████████████████████████| 11.6 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 16.9 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [3]:
!ls food-101-subset

images	license_agreement.txt  readme.txt


In [4]:
!ls food-101-subset/images

apple_pie  caesar_salad  falafel


In [36]:
image_size = (224, 224)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "food-101-subset/images",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    labels='inferred',
    label_mode='categorical',
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "food-101-subset/images",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    labels='inferred',
    label_mode='categorical',
)

Found 3000 files belonging to 3 classes.
Using 2400 files for training.
Found 3000 files belonging to 3 classes.
Using 600 files for validation.


In [23]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [38]:
augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

In [39]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [60]:
%%time
base_model_vgg19 = VGG19(weights='imagenet', include_top=False)
x = base_model_vgg19.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
preds = layers.Dense(3, activation='softmax')(x)
model_vgg19 = Model(inputs=base_model_vgg19.input, outputs=preds)

# freeze
for layer in base_model_vgg19.layers:
    layer.trainable = False
model_vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg19.fit(augmented_train_ds, epochs=5, validation_data=val_ds)

#fine-tune
for layer in base_model_vgg19.layers:
    layer.trainable = True
    
opt = keras.optimizers.Adam(learning_rate=0.00001)
model_vgg19.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg19.fit(augmented_train_ds, epochs=20, validation_data=val_ds)

Epoch 1/5
75/75 [==============================] - 24s 306ms/step - loss: 2.5921 - accuracy: 0.6306 - val_loss: 0.6047 - val_accuracy: 0.8450
Epoch 2/5
75/75 [==============================] - 19s 251ms/step - loss: 0.3596 - accuracy: 0.8710 - val_loss: 0.6480 - val_accuracy: 0.8317
Epoch 3/5
75/75 [==============================] - 20s 255ms/step - loss: 0.2911 - accuracy: 0.8991 - val_loss: 0.4750 - val_accuracy: 0.8550
Epoch 4/5
75/75 [==============================] - 19s 254ms/step - loss: 0.2808 - accuracy: 0.8960 - val_loss: 0.5087 - val_accuracy: 0.8617
Epoch 5/5
75/75 [==============================] - 19s 254ms/step - loss: 0.2077 - accuracy: 0.9191 - val_loss: 0.4438 - val_accuracy: 0.8700
Epoch 1/20
75/75 [==============================] - 29s 368ms/step - loss: 0.2104 - accuracy: 0.9259 - val_loss: 0.3951 - val_accuracy: 0.8833
Epoch 2/20
75/75 [==============================] - 28s 365ms/step - loss: 0.1162 - accuracy: 0.9581 - val_loss: 0.3788 - val_accuracy: 0.9000
Epoc

In [61]:
%%time
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False)
x = base_model_resnet50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
preds = layers.Dense(3, activation='softmax')(x)
model_resnet50 = Model(inputs=base_model_resnet50.input, outputs=preds)

# freeze
for layer in base_model_resnet50.layers:
    layer.trainable = False
model_resnet50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet50.fit(augmented_train_ds, epochs=5, validation_data=val_ds)

#fine-tune
for layer in base_model_resnet50.layers:
    layer.trainable = True
    
opt = keras.optimizers.Adam(learning_rate=0.00001)
model_resnet50.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet50.fit(augmented_train_ds, epochs=20, validation_data=val_ds)

Epoch 1/5
75/75 [==============================] - 19s 224ms/step - loss: 1.9389 - accuracy: 0.6095 - val_loss: 0.3630 - val_accuracy: 0.8633
Epoch 2/5
75/75 [==============================] - 17s 215ms/step - loss: 0.3243 - accuracy: 0.8829 - val_loss: 0.3387 - val_accuracy: 0.8650
Epoch 3/5
75/75 [==============================] - 16s 214ms/step - loss: 0.2352 - accuracy: 0.9139 - val_loss: 0.3159 - val_accuracy: 0.8750
Epoch 4/5
75/75 [==============================] - 17s 215ms/step - loss: 0.2027 - accuracy: 0.9230 - val_loss: 0.2876 - val_accuracy: 0.8833
Epoch 5/5
75/75 [==============================] - 17s 216ms/step - loss: 0.1875 - accuracy: 0.9285 - val_loss: 0.2367 - val_accuracy: 0.9083
Epoch 1/20
75/75 [==============================] - 26s 301ms/step - loss: 0.2963 - accuracy: 0.8730 - val_loss: 0.2466 - val_accuracy: 0.9000
Epoch 2/20
75/75 [==============================] - 22s 290ms/step - loss: 0.1553 - accuracy: 0.9429 - val_loss: 0.2015 - val_accuracy: 0.9117
Epoc

In [62]:
%%time
base_model_mobilenetv2 = MobileNetV2(weights='imagenet', include_top=False)
x = base_model_mobilenetv2.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
preds = layers.Dense(3, activation='softmax')(x)
model_mobilenetv2 = Model(inputs=base_model_mobilenetv2.input, outputs=preds)

# freeze
for layer in base_model_mobilenetv2.layers:
    layer.trainable = False
model_mobilenetv2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_mobilenetv2.fit(augmented_train_ds, epochs=5, validation_data=val_ds)

#fine-tune
for layer in base_model_mobilenetv2.layers:
    layer.trainable = True
    
opt = keras.optimizers.Adam(learning_rate=0.00001)
model_mobilenetv2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model_mobilenetv2.fit(augmented_train_ds, epochs=20, validation_data=val_ds)

Epoch 1/5
75/75 [==============================] - 17s 199ms/step - loss: 1.4948 - accuracy: 0.4741 - val_loss: 0.8183 - val_accuracy: 0.6317
Epoch 2/5
75/75 [==============================] - 15s 195ms/step - loss: 0.8243 - accuracy: 0.6211 - val_loss: 0.8300 - val_accuracy: 0.5967
Epoch 3/5
75/75 [==============================] - 15s 192ms/step - loss: 0.7894 - accuracy: 0.6490 - val_loss: 0.9157 - val_accuracy: 0.5817
Epoch 4/5
75/75 [==============================] - 15s 193ms/step - loss: 0.7730 - accuracy: 0.6480 - val_loss: 0.7358 - val_accuracy: 0.6817
Epoch 5/5
75/75 [==============================] - 15s 192ms/step - loss: 0.7543 - accuracy: 0.6781 - val_loss: 0.8056 - val_accuracy: 0.6467
Epoch 1/20
75/75 [==============================] - 24s 282ms/step - loss: 2.6102 - accuracy: 0.3714 - val_loss: 3.0680 - val_accuracy: 0.3667
Epoch 2/20
75/75 [==============================] - 21s 274ms/step - loss: 0.7391 - accuracy: 0.6761 - val_loss: 2.9911 - val_accuracy: 0.3850
Epoc

In [57]:
model_vgg19.save('model_vgg19.h5')

In [58]:
model_resnet50.save('model_resnet50.h5')

In [59]:
model_mobilenetv2.save('model_mobilenetv2.h5')

In [63]:
!ls -l *.h5

-rw-r--r-- 1 1024 users  43463592 Mar  5 11:40 model_mobilenetv2.h5
-rw-r--r-- 1 1024 users 308839504 Mar  5 11:39 model_resnet50.h5
-rw-r--r-- 1 1024 users 246805616 Mar  5 11:39 model_vgg19.h5


In [64]:
model_vgg19.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0  

In [65]:
model_resnet50.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_18[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [66]:
model_mobilenetv2.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         input_19[0][0]                   
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, None, 3 0           bn_Conv1[0][0]                   
___________________________________________________________________________________________